In [1]:
!pip install -q transformers datasets faiss-gpu # Or faiss-cpu
!git clone https://github.com/louis030195/big-talks

     |████████████████████████████████| 1.8MB 16.4MB/s 
     |████████████████████████████████| 163kB 37.5MB/s 
     |████████████████████████████████| 89.4MB 51kB/s 
     |████████████████████████████████| 2.9MB 51.9MB/s 
     |████████████████████████████████| 890kB 62.3MB/s 
     |████████████████████████████████| 245kB 59.3MB/s 
     |████████████████████████████████| 20.7MB 1.3MB/s 
Cloning into 'big-talks'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 21 (delta 3), reused 15 (delta 1), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [15]:
import os
import pandas as pd
import requests
import datetime
from datasets import Dataset
df = pd.read_json("big-talks/docs/big-talks.json")
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['basic'],
    num_rows: 44
})

In [19]:
dataset.map(lambda s: print(s["basic"]))

What insight about life have you acquired, that seems obvious to you but might not be obvious to everyone else?


What insight about life have you acquired, that seems obvious to you but might not be obvious to everyone else?
In only one sentence, how would you explain the fundamentals of your profession that would contain the most information in the fewest words possible?
If you could have anyone (dead or alive) on your personal board of directors what 1-3 people would you choose?
When was the last time you changed your mind about something?
Name a writer or publication you disagree with but still read.
What single book is the best introduction to your field (or specialization within your field) for laypeople? 
What do you look for in a very close friend?
What have you learned about yourself in the past year?
In what ways is your family especially odd?
What book have you recommended the most?
Which one of your friends do you envy the most?
If you were braver, what would you be doing with your life?
What’s your worst intellectual habit?
What’s a view that you hold but can’t defend?
Would you rathe

Dataset({
    features: ['basic'],
    num_rows: 44
})

In [17]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
torch.set_grad_enabled(False)
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
dataset_with_embeddings = dataset.map(lambda s: {'embeddings': ctx_encoder(**ctx_tokenizer(s["basic"], return_tensors="pt"))[0][0].numpy()})
dataset_with_embeddings.add_faiss_index(column='embeddings')

Dataset({
    features: ['basic', 'embeddings'],
    num_rows: 44
})

In [21]:
quote_embedding = ctx_encoder(**ctx_tokenizer("What did school help to kill in you?", return_tensors="pt"))[0][0].numpy()
dataset_with_embeddings.get_nearest_examples('embeddings', quote_embedding, k=10)

NearestExamplesResults(scores=array([ 0.      , 37.617718, 44.557613, 45.990013, 47.45672 , 47.98828 ,
       50.85805 , 51.04663 , 52.86026 , 53.319313], dtype=float32), examples=OrderedDict([('basic', ['What did school help to kill in you?', 'What do other people not understand about you?', 'When was the last time you changed your mind about something?', 'What would you want to correct about your school or university education?', 'What’s your worst intellectual habit?', 'What do you need to believe in order to get through the day?', 'What have you learned about yourself in the past year?', 'If you had to write a book, what would it be about?', 'Which one of your friends do you envy the most?', 'If you were braver, what would you be doing with your life?']), ('embeddings', [[0.33822259306907654, -0.06435135006904602, -0.22143903374671936, 0.4183570146560669, -0.4046576917171478, 0.07075648754835129, 0.19269335269927979, 0.07952296733856201, -0.3306245803833008, 0.10905330628156662, 0.

In [36]:
def nearest_big_talk(big_talk):
    big_talk_embedding = ctx_encoder(**ctx_tokenizer(big_talk, return_tensors="pt"))[0][0].numpy()
    scores, retrieved_examples = dataset_with_embeddings.get_nearest_examples('embeddings', big_talk_embedding, k=10)
    return retrieved_examples['basic']

big_talks = [
    """If you could have anyone (dead or alive) on your personal board of directors what 1-3 people would you choose?""",
]

for big_talk in big_talks:
  print("big talk:", big_talk)
  print("nearest big talks:")
  [print(" - ", n_q) for n_q in nearest_big_talk(big_talk)]
  print("~~~~~~~~~~~~~")


big talk: If you could have anyone (dead or alive) on your personal board of directors what 1-3 people would you choose?
nearest big talks:
 -  If you could have anyone (dead or alive) on your personal board of directors what 1-3 people would you choose?
 -  In only one sentence, how would you explain the fundamentals of your profession that would contain the most information in the fewest words possible?
 -  If you had to save three things from your house (Other than pets or people) what would you take?
 -  Let’s say a young person comes to you and says, “I want to be the next you, not exactly a duplicate of what you do but some version of what that will be 30 years from now.” what skill do you tell them to invest in?
 -  What single book is the best introduction to your field (or specialization within your field) for laypeople? 
 -  What insight about life have you acquired, that seems obvious to you but might not be obvious to everyone else?
 -  When you feel upset, how - ideally - 

In [25]:
import json
import requests
API_TOKEN = os.environ["API_TOKEN"]
API_URL = "https://api-inference.huggingface.co/models/gpt2"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))

data = query("Can you please let us know more details about your ")

In [26]:
data

[{'generated_text': "Can you please let us know more details about your iphone?\n\nIf you can't find your iphone in the store, we won't be able to match it with yours.\n\nUnfortunately, many customers don't get the"}]

In [34]:
data = [query(e) for e in df.basic]

In [35]:
data

[[{'generated_text': "What insight about life have you acquired, that seems obvious to you but might not be obvious to everyone else? What insight does it take to become happy?\n\n1. It's easy to get happy. You've already done some positive things;"}],
 [{'generated_text': 'In only one sentence, how would you explain the fundamentals of your profession that would contain the most information in the fewest words possible? How would they tell you which class would be most effective in your profession? Would you even be able to answer these'}],
 [{'generated_text': 'If you could have anyone (dead or alive) on your personal board of directors what 1-3 people would you choose?\n\nIf I could choose just one person, what would it be like with my 2 people and their 2 children?\n'}],
 [{'generated_text': 'When was the last time you changed your mind about something?\n\n\nWell, I still remember the first time I did it. I wrote that one-word thing to my mom after the first time I saw her -- the k